<a href="https://colab.research.google.com/github/Cat-sandwich/Labochka_5/blob/main/Labochka_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import main
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from pymystem3 import Mystem

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Пункт 1

Необходимо загрузить исходный набор данных и соответствующие метки классов.

In [100]:
data = pd.read_csv('dataframe.csv')
data.head()

,class_mark,text_review
0,good,Зеленая миля\n'Они помогли ему убить себя и та...
1,good,"Зеленая миля\n'Боже, иногда Зеленая миля так д..."
2,good,Зеленая миля\nОни помогли ему убить себя\n\n«О...
3,good,Зеленая миля\n«Когда призовет Господь на свой ...
4,good,Зеленая миля\nОбвиненный в страшном преступлен...


In [101]:
remove_non_alphabets =lambda x: re.sub(r'[^а-яА-Я]',' ',x)

In [102]:
tokenize = lambda x: word_tokenize(x, language="russian")

In [103]:
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

In [104]:
lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

In [105]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [106]:
def Lemmatize(src: list):
    ''' Лемматизирует переданный датасет
    '''
    text_nomalized = ' '.join(src).lower() 

    m = Mystem()
    lemmas = m.lemmatize(text_nomalized)
    
    return lemmas

In [107]:
print('Processing : [=', end='')

data['text_review'] = data['text_review'].apply(remove_non_alphabets)
print('=', end='')
data['text_review'] = data['text_review'].apply(tokenize) # [ word_tokenize(row) for row in data['email']]
print('=', end='')
data['text_review'] = data['text_review'].apply(stem)
data['text_review'] = data['text_review'].apply(leammtizer)

data['text_review'] = data['text_review'].apply(lambda x: ' '.join(x))
print('=] : Completed', end='')
data.head()


Processing : [====] : Completed

,class_mark,text_review
0,good,зеленая миля они помогли ему убить себя и так ...
1,good,зеленая миля боже иногда зеленая миля так длин...
2,good,зеленая миля они помогли ему убить себя они по...
3,good,зеленая миля когда призовет господь на свой су...
4,good,зеленая миля обвиненный в страшном преступлени...


In [108]:
#max_words = 10000
bad_words = stopwords.words("russian")
cv = CountVectorizer(stop_words=bad_words)
sparse_matrix = cv.fit_transform(data['text_review']).toarray()

In [114]:
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(data['class_mark']), test_size = 0.1, train_size = 0.8)

In [115]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [111]:
model = LogisticRegression()

In [112]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)

In [113]:
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.